In [0]:
# import libraries
import pandas as pd
import os
import glob
import joblib
from sklearn.preprocessing import MinMaxScaler

In [0]:
# boolean to indicate if this is the initial dataset
# used to determine whether scaler should be generated or loaded from file
initial_dataset = True

In [0]:
# import hourly weather data and create dataframe
all_files = glob.glob(os.path.join('data/weather_data/all_hourly_data/', "*.csv")) 
df_from_each_file = (pd.read_csv(f, skiprows=15) for f in all_files)
df_hourly = pd.concat(df_from_each_file, ignore_index=True)

In [0]:
df_hourly.head()

,Date/Time,Year,Month,Day,Time,Temp (°C),Temp Flag,Dew Point Temp (°C),Dew Point Temp Flag,Rel Hum (%),...,Wind Spd Flag,Visibility (km),Visibility Flag,Stn Press (kPa),Stn Press Flag,Hmdx,Hmdx Flag,Wind Chill,Wind Chill Flag,Weather
0,2014-10-01 00:00,2014,10,1,00:00,12.7,NaN,10.1,NaN,85.0,...,NaN,NaN,NaN,101.47,NaN,NaN,NaN,NaN,NaN,NaN
1,2014-10-01 01:00,2014,10,1,01:00,12.5,NaN,10.0,NaN,85.0,...,NaN,NaN,NaN,101.49,NaN,NaN,NaN,NaN,NaN,NaN
2,2014-10-01 02:00,2014,10,1,02:00,12.2,NaN,9.5,NaN,84.0,...,NaN,NaN,NaN,101.46,NaN,NaN,NaN,NaN,NaN,NaN
3,2014-10-01 03:00,2014,10,1,03:00,12.1,NaN,9.5,NaN,84.0,...,NaN,NaN,NaN,101.49,NaN,NaN,NaN,NaN,NaN,NaN
4,2014-10-01 04:00,2014,10,1,04:00,11.9,NaN,9.4,NaN,85.0,...,NaN,NaN,NaN,101.53,NaN,NaN,NaN,NaN,NaN,NaN


In [0]:
# drop columns that aren't needed
df_hourly.drop(['Year', 'Month', 'Day', 'Time', 'Temp Flag', 'Dew Point Temp (°C)', 'Dew Point Temp Flag', 
                'Rel Hum Flag', 'Wind Dir (10s deg)',
                'Wind Dir Flag', 'Wind Spd Flag',  'Visibility (km)', 'Visibility Flag',
                'Stn Press Flag', 'Hmdx', 'Hmdx Flag', 'Wind Chill', 'Wind Chill Flag', 'Weather'], 
               axis=1, inplace=True)

In [0]:
# convert to datetime
df_hourly['Date/Time'] =  pd.to_datetime(df_hourly['Date/Time'], infer_datetime_format=True)

In [0]:
# use Date/Time to generate Month, DOW, Hour, and Date column
df_hourly['Month'] =  df_hourly['Date/Time'].dt.month
df_hourly['DayOfWeek'] =  df_hourly['Date/Time'].dt.dayofweek
df_hourly['HourOfDay'] =  df_hourly['Date/Time'].dt.hour
df_hourly['Date'] =  df_hourly['Date/Time'].dt.date
# rename columns
df_hourly.rename(index=str, columns={"Temp (°C)": "Temp", "Rel Hum (%)": "RelHum", 
                                     "Wind Spd (km/h)": "WindSpd",
                                    "Stn Press (kPa)": "Pressure"}, inplace=True)

In [0]:
# add a column to indicate 'Time of Day' - bracket into 6 hr brackets
tod_bins = [0, 6, 12, 18, 24]
tod_labels = ['Night', 'Morning','Afternoon','Evening']
hours = df_hourly['Date/Time'].dt.hour
df_hourly['TimeOfDay'] = pd.cut(hours, bins=tod_bins, labels=tod_labels, right=False)
df_hourly["TimeOfDay"] = df_hourly["TimeOfDay"].astype('category')
df_hourly["TimeOfDay"] = df_hourly["TimeOfDay"].cat.codes
# drop Date/Time now that it's not required
df_hourly.drop(['Date/Time'], axis=1, inplace=True)

In [0]:
# load daily weather data and create dataframe
all_files = glob.glob(os.path.join('data/weather_data/all_daily_data/', "*.csv"))
df_from_each_file = (pd.read_csv(f, skiprows=24) for f in all_files)
df_daily = pd.concat(df_from_each_file, ignore_index=True)

In [0]:
#create new feature - cumulative rainfall over 5 days
df_daily['TotalPrecip_1'] = df_daily['Total Precip (mm)'].shift(1)
df_daily['TotalPrecip_2'] = df_daily['Total Precip (mm)'].shift(2)
df_daily['TotalPrecip_3'] = df_daily['Total Precip (mm)'].shift(3)
df_daily['TotalPrecip_4'] = df_daily['Total Precip (mm)'].shift(4)
df_daily['CumulPrecip'] = df_daily[['Total Precip (mm)', 'TotalPrecip_1', 
                                    'TotalPrecip_2', 'TotalPrecip_3', 
                                    'TotalPrecip_4']].sum(axis=1)

In [0]:
# drop columns that aren't required
df_daily.drop(['Year', 'Month', 'Day', 'Data Quality', 'Max Temp (°C)', 'Max Temp Flag', 'Min Temp (°C)', 
               'Min Temp Flag', 'Mean Temp Flag', 'Mean Temp (°C)',
               'Total Snow Flag', 'Total Rain Flag', 'Cool Deg Days Flag', 'Heat Deg Days Flag', 
               'Dir of Max Gust (10s deg)', 'Cool Deg Days (°C)', 'Heat Deg Days (°C)', 
               'Dir of Max Gust Flag', 'Snow on Grnd Flag', 'Spd of Max Gust Flag', 'Total Precip Flag',
               'Spd of Max Gust (km/h)', 'Total Rain (mm)', 'Total Snow (cm)',
              'TotalPrecip_1', 'TotalPrecip_2', 'TotalPrecip_3', 'TotalPrecip_4'],
              axis=1, inplace=True)

In [0]:
# rename columns
df_daily.rename(index=str, columns={"Total Precip (mm)": "TotalPrecip", "Snow on Grnd (cm)": "SnowOnGrnd"},
                inplace=True)

In [0]:
# generate date from Date/Time so we can join with hourly dataset
df_daily['Date/Time'] =  pd.to_datetime(df_daily['Date/Time'], infer_datetime_format=True)
df_daily['Date'] =  df_daily['Date/Time'].dt.date

In [0]:
# drop Date/Time as it's no longer needed
df_daily.drop(['Date/Time'], axis=1, inplace=True)

In [0]:
# merge datasets
df = pd.merge(df_hourly, df_daily, on='Date')
df.head()

,Temp,RelHum,WindSpd,Pressure,Month,DayOfWeek,HourOfDay,Date,TimeOfDay,TotalPrecip,SnowOnGrnd,CumulPrecip
0,12.7,85.0,15.0,101.47,10,2,0,2014-10-01,0,NaN,NaN,0.0
1,12.5,85.0,19.0,101.49,10,2,1,2014-10-01,0,NaN,NaN,0.0
2,12.2,84.0,17.0,101.46,10,2,2,2014-10-01,0,NaN,NaN,0.0
3,12.1,84.0,14.0,101.49,10,2,3,2014-10-01,0,NaN,NaN,0.0
4,11.9,85.0,15.0,101.53,10,2,4,2014-10-01,0,NaN,NaN,0.0


In [0]:
# examine NA values
df.isna().sum()

Temp            5086
RelHum          5084
WindSpd         5110
Pressure        5087
Month              0
DayOfWeek          0
HourOfDay          0
Date               0
TimeOfDay          0
TotalPrecip     6288
SnowOnGrnd     40488
CumulPrecip        0
dtype: int64

In [0]:
# replace NA values by monthly averages
columns_to_average = ['Temp', 'RelHum', 'WindSpd', 'TotalPrecip', 'Pressure', 'SnowOnGrnd']

for column in columns_to_average:
    df[column].fillna(df.groupby("Month")[column].transform("mean"), inplace=True)

In [0]:
# average weather variables over the Time of Day brackets for each day
df_weather = df.groupby(['Date', 'TimeOfDay'], as_index=False)['Temp', 'RelHum', 'WindSpd', 
                                                               'TotalPrecip', 'CumulPrecip',
                                                               'SnowOnGrnd', 'Pressure'].mean()

In [0]:
# add a new feature - Wind Power
# Wind Power is proportional to Pressure, Wind Speed ^3, and 1/Temperature
df_weather['WindPwr'] = df_weather['Pressure'] * df_weather['WindSpd']**3 / (df_weather['Temp'] + 273.15)

In [0]:
df_weather.head()

,Date,TimeOfDay,Temp,RelHum,WindSpd,TotalPrecip,CumulPrecip,SnowOnGrnd,Pressure,WindPwr
0,2005-01-01,0,5.916667,73.500000,35.666667,0.2,2.0,2.0,100.753333,16380.947832
1,2005-01-01,1,2.283333,73.000000,27.166667,0.2,2.0,2.0,101.716667,7404.311540
2,2005-01-01,2,-1.250000,56.000000,20.833333,0.2,2.0,2.0,102.613333,3412.485981
3,2005-01-01,3,-6.583333,53.166667,19.333333,0.2,2.0,2.0,103.448333,2804.386536
4,2005-01-02,0,-13.250000,63.000000,21.500000,11.4,12.7,1.0,103.791667,3968.913064


In [0]:
columns_to_scale = ['Temp', 'RelHum', 'WindSpd', 'TotalPrecip', 'CumulPrecip', 'SnowOnGrnd', 'WindPwr', 'Pressure']

In [0]:
if initial_dataset:
    # generate new scaler for initial dataset
    scaler = MinMaxScaler(feature_range=(0, 1), copy=True)
    df_weather[columns_to_scale] = scaler.fit_transform(df_weather[columns_to_scale])
    joblib.dump(scaler, 'scaler.pkl')
else:
    # load previous scaler for new data
    scaler = joblib.load('scaler.pkl')
    df_weather[columns_to_scale] = scaler.fit_transform(df_weather[columns_to_scale])

In [0]:
df_weather.head()

,Date,TimeOfDay,Temp,RelHum,WindSpd,TotalPrecip,CumulPrecip,SnowOnGrnd,Pressure,WindPwr
0,2005-01-01,0,0.533699,0.689453,0.569149,0.002710,0.016447,0.042553,0.506920,0.174991
1,2005-01-01,1,0.473973,0.683594,0.433511,0.002710,0.016447,0.042553,0.649778,0.079097
2,2005-01-01,2,0.415890,0.484375,0.332447,0.002710,0.016447,0.042553,0.782748,0.036454
3,2005-01-01,3,0.328219,0.451172,0.308511,0.002710,0.016447,0.042553,0.906574,0.029958
4,2005-01-02,0,0.218630,0.566406,0.343085,0.154472,0.104441,0.021277,0.957489,0.042398


In [0]:
# scaler can be used to inverse transform the data
#df_weather[columns_to_scale] = scaler.inverse_transform(df_weather[columns_to_scale])
#df_weather.head()

In [0]:
#write to file
df_weather.to_pickle('weather_data.pkl')